In [ ]:
!sudo apt install graphviz
!pip install pydot
!pip install onnx
!git clone https://github.com/onnx/onnx.git
!pip install onnxruntime-gpu
!pip install onnx_tf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
     |████████████████████████████████| 7.7MB 11.4MB/s 
Cloning into 'onnx'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 25847 (delta 14), reused 19 (delta 0), pack-reused 25756
Receiving objects: 100% (25847/25847), 18.99 MiB | 25.90 MiB/s, done.
Resolving deltas: 100% (14107/14107), done.
     |████████████████████████████████| 30.9MB 161kB/s 
     |████████████████████████████████| 215kB 14.6MB/s 


In [ ]:
import torch
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torchvision.models.alexnet(pretrained=True).to(device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [ ]:
dummy_input = torch.randn(10, 3, 224, 224).to(device)
input_names = [ "input_1" ] + [f"learned_{i}" for i in range(16)]
output_names = ["output1"]

In [ ]:
torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)

graph(%input_1 : Float(10:150528, 3:50176, 224:224, 224:1, requires_grad=0, device=cuda:0),
      %learned_0 : Float(64:363, 3:121, 11:11, 11:1, requires_grad=1, device=cuda:0),
      %learned_1 : Float(64:1, requires_grad=1, device=cuda:0),
      %learned_2 : Float(192:1600, 64:25, 5:5, 5:1, requires_grad=1, device=cuda:0),
      %learned_3 : Float(192:1, requires_grad=1, device=cuda:0),
      %learned_4 : Float(384:1728, 192:9, 3:3, 3:1, requires_grad=1, device=cuda:0),
      %learned_5 : Float(384:1, requires_grad=1, device=cuda:0),
      %learned_6 : Float(256:3456, 384:9, 3:3, 3:1, requires_grad=1, device=cuda:0),
      %learned_7 : Float(256:1, requires_grad=1, device=cuda:0),
      %learned_8 : Float(256:2304, 256:9, 3:3, 3:1, requires_grad=1, device=cuda:0),
      %learned_9 : Float(256:1, requires_grad=1, device=cuda:0),
      %learned_10 : Float(4096:9216, 9216:1, requires_grad=1, device=cuda:0),
      %learned_11 : Float(4096:1, requires_grad=1, device=cuda:0),
      %learne

In [ ]:
!python onnx/onnx/tools/net_drawer.py --input alexnet.onnx --output alexnet.dot --embed_docstring

In [ ]:
!dot -Tsvg alexnet.dot -o alexnet.svg

https://pytorch.org/docs/stable/_modules/torchvision/models/alexnet.html#alexnet

In [ ]:
import onnx
from onnx_tf.backend import prepare

In [ ]:
model_onnx = onnx.load("alexnet.onnx")

In [ ]:
onnx.checker.check_model(model_onnx)

In [ ]:
print(onnx.helper.printable_graph(model_onnx.graph))

graph torch-jit-export (
  %input_1[FLOAT, 10x3x224x224]
) initializers (
  %learned_0[FLOAT, 64x3x11x11]
  %learned_1[FLOAT, 64]
  %learned_10[FLOAT, 4096x9216]
  %learned_11[FLOAT, 4096]
  %learned_12[FLOAT, 4096x4096]
  %learned_13[FLOAT, 4096]
  %learned_14[FLOAT, 1000x4096]
  %learned_15[FLOAT, 1000]
  %learned_2[FLOAT, 192x64x5x5]
  %learned_3[FLOAT, 192]
  %learned_4[FLOAT, 384x192x3x3]
  %learned_5[FLOAT, 384]
  %learned_6[FLOAT, 256x384x3x3]
  %learned_7[FLOAT, 256]
  %learned_8[FLOAT, 256x256x3x3]
  %learned_9[FLOAT, 256]
) {
  %17 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%input_1, %learned_0, %learned_1)
  %18 = Relu(%17)
  %19 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%18)
  %20 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2, 2], strides = [1, 1]](%19, %learned_2, %learned_3)
  %21 = Relu(%20)
  %22 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], st

In [ ]:
tf_rep = prepare(model_onnx)

In [ ]:
export_dir = "tf_model"
tf_rep.export_graph(export_dir)

INFO:tensorflow:Assets written to: tf_model/assets


INFO:tensorflow:Assets written to: tf_model/assets


In [ ]:
import tensorflow as tf

In [ ]:
imported = tf.saved_model.load(export_dir)
f = imported.signatures["serving_default"]

In [ ]:
imported.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input_1) at 0x7F9C2A0A4CC0>}))

In [ ]:
out = f(input_1=dummy_input.cpu())

In [ ]:
out["output_0"]

<tf.Tensor: shape=(10, 1000), dtype=float32, numpy=
array([[-0.02179407, -1.2334436 , -0.9728871 , ..., -1.1794727 ,
        -1.0076256 ,  1.1060724 ],
       [-0.06033897, -1.1013155 , -1.2295612 , ..., -1.0795169 ,
        -1.0598122 ,  1.0903145 ],
       [ 0.0451268 , -1.2982653 , -1.0168775 , ..., -1.3562102 ,
        -0.9385383 ,  0.9475912 ],
       ...,
       [ 0.25677   , -1.0557717 , -0.9824258 , ..., -0.9874987 ,
        -1.051997  ,  1.3452493 ],
       [ 0.10032231, -1.1367997 , -1.591173  , ..., -1.5932413 ,
        -0.9429994 ,  1.2389797 ],
       [-0.30203348, -1.7088867 , -2.2362735 , ..., -1.5673717 ,
        -1.1156543 ,  1.1354088 ]], dtype=float32)>

____________

In [ ]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=True)

example = torch.rand(1, 3, 224, 224)

traced_script_module = torch.jit.trace(model, example)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
output = traced_script_module(torch.ones(1, 3, 224, 224))

In [ ]:
print(output)

tensor([[-6.4706e-01, -5.6287e-01, -4.5664e-01, -1.4785e+00, -7.0649e-01,
         -1.0570e-01, -5.2827e-01,  4.8727e-01,  2.5138e-01, -8.6397e-01,
         -1.0303e+00, -8.8330e-01, -3.4483e-01, -9.7994e-01, -1.1171e+00,
         -6.4003e-01, -7.8201e-01, -1.6677e-01, -5.0852e-01, -5.5767e-01,
         -1.5480e+00, -6.6745e-01, -1.4056e+00,  1.3023e-01, -9.0067e-01,
         -1.1809e+00, -9.3536e-01, -1.2858e+00, -8.2091e-01, -4.4468e-01,
         -9.3707e-01, -8.8239e-01, -6.0408e-01, -4.8926e-01, -2.9906e-01,
         -4.2005e-01,  5.0769e-01, -7.4372e-01, -4.1432e-01,  2.8811e-02,
         -7.5651e-01, -9.3893e-01, -1.1775e+00, -4.3256e-01, -6.5951e-01,
         -4.1137e-01, -7.8986e-01, -4.7914e-01, -1.3821e+00, -1.1447e+00,
         -4.6387e-01,  2.9823e-01, -4.1075e-01, -6.3856e-01, -1.4058e-01,
         -1.2304e+00, -3.7980e-01, -1.5105e+00, -6.2983e-01, -5.1889e-01,
          7.6164e-01,  1.8636e-01, -1.9671e-01,  1.6489e-01, -7.0234e-01,
         -2.3402e-01, -3.2406e-01, -2.

In [ ]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, inp):
        if inp.sum() > 0:
            output = self.weight.mv(inp)
        else:
            output = self.weight + inp
        return output

In [ ]:
my_module = MyModule(20, 30)
sm = torch.jit.script(my_module)

In [ ]:
traced_script_module.save("traced.pt")
sm.save("sm.pt")